# Lab 07 - Convolutional Neural Networks

Step 1: Install the necessary libraries

In [ ]:
%pip install torch torchvision

Step 2: Import required libraries and their functions

In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torchvision
import torchvision.transforms as transforms 
from torchvision import models
from torch.utils.data import DataLoader


Step 3: Load and Preprocess the MNIST Dataset

In [3]:
transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



Step 4: Load the Pre-trained VGG Model and Modify It

In [3]:
class MNISTResNet(nn.Module):
    def __init__(self):
        super(MNISTResNet, self).__init__()
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)

    def forward(self, x):
        x = self.resnet(x)
        return x

class MNISTAlexNet(nn.Module):
    def __init__(self):
        super(MNISTAlexNet, self).__init__()
        self.alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
        self.alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
        num_ftrs = self.alexnet.classifier[6].in_features
        self.alexnet.classifier[6] = nn.Linear(num_ftrs, 10)

    def forward(self, x):
        x = self.alexnet(x)
        return x

Step 5: Define the Loss Function and optimizer and initialize the model

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

resnet_model = MNISTResNet().to(device)
alexnet_model = MNISTAlexNet().to(device)

criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
alexnet_optimizer = optim.Adam(alexnet_model.parameters(), lr=0.001)


Step 6: Train the Model

In [7]:
def train_model(model, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print(f"internel Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total} %')

train_model(resnet_model, resnet_optimizer)
evaluate_model(resnet_model)

train_model(alexnet_model, alexnet_optimizer)
evaluate_model(alexnet_model)

internel Epoch [1/10], Loss: 0.0013
internel Epoch [1/10], Loss: 0.0021
internel Epoch [1/10], Loss: 0.0027
internel Epoch [1/10], Loss: 0.0032
internel Epoch [1/10], Loss: 0.0035
internel Epoch [1/10], Loss: 0.0039
internel Epoch [1/10], Loss: 0.0040
internel Epoch [1/10], Loss: 0.0041
internel Epoch [1/10], Loss: 0.0043
internel Epoch [1/10], Loss: 0.0044
internel Epoch [1/10], Loss: 0.0047
internel Epoch [1/10], Loss: 0.0048
internel Epoch [1/10], Loss: 0.0050
internel Epoch [1/10], Loss: 0.0052
internel Epoch [1/10], Loss: 0.0053
internel Epoch [1/10], Loss: 0.0054
internel Epoch [1/10], Loss: 0.0055
internel Epoch [1/10], Loss: 0.0056
internel Epoch [1/10], Loss: 0.0057
internel Epoch [1/10], Loss: 0.0058
internel Epoch [1/10], Loss: 0.0060
internel Epoch [1/10], Loss: 0.0061
internel Epoch [1/10], Loss: 0.0065
internel Epoch [1/10], Loss: 0.0067
internel Epoch [1/10], Loss: 0.0069
internel Epoch [1/10], Loss: 0.0069
internel Epoch [1/10], Loss: 0.0071
internel Epoch [1/10], Loss:

KeyboardInterrupt: 